In [13]:
import os
import pandas as pd
from music21 import converter, chord, note

In [17]:
from music21 import converter, note, chord
import pandas as pd
import os

# Function to clean chord names
def clean_chord_name(name):
    # Remove common suffixes in chord names
    for suffix in ['chord', 'triad']:
        name = name.replace(suffix, '')
    return name.strip()

# Adjusted function to extract and clean notes and chords
def extract_notes_and_chords(midifile):
    midi_data = converter.parse(midifile)
    notes_and_chords = []

    for element in midi_data.recurse():
        if isinstance(element, note.Note):
            # Only add note if it's not in the lower octave (e.g., C2)
            if element.octave > 2:
                notes_and_chords.append({"Type": "Note", "Content": str(element.pitch)})
        elif isinstance(element, chord.Chord):
            # Filter out lower octave notes from the chord
            filtered_notes = [n for n in element.pitches if n.octave > 2]
            chord_notes = ' '.join(n.nameWithOctave for n in filtered_notes)
            if chord_notes:  # Only proceed if there are notes left after filtering
                chord_name = clean_chord_name(element.pitchedCommonName)
                notes_and_chords.append({"Type": "Chord", "Name": chord_name, "Notes": chord_notes})

    return notes_and_chords

# Function to save the extracted notes and chords, including cleaned chord names, to a CSV file
def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

# Update these paths according to your setup
midi_dir = '$'  # Update this path
output_csv = 'notes_chords_cleaned.csv'  # Output file path

all_elements_data = []

for root, dirs, files in os.walk(midi_dir):
    for file in files:
        if file.lower().endswith(('.mid', '.midi')):
            midi_path = os.path.join(root, file)
            elements = extract_notes_and_chords(midi_path)
            all_elements_data.extend(elements)

save_to_csv(all_elements_data, output_csv)


Data saved to notes_chords_cleaned.csv
